In [ ]:
#save and load simulations
#martinize posres erhöhen?

In [1]:
%load_ext autoreload

import encodermap as em
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
#import gromacs
import glob2 as glob
import os
import sys
import pickle
import math
import subprocess
import seaborn
import pandas as pd
import random
import matplotlib.cm as cm
import pyemma.plots as mplt
import scipy as sp
sys.path.insert(1, "/home/nicolas.schneider/nicolas/master_thesis/backward-v5")
from collections import defaultdict
import pprint
sys.path.insert(1, '/home/nicolas.schneider/soft/running_rabbit')


from sim_class import *
from utility_functions import *

In [ ]:
%autoreload

In [2]:
cg_simulation_dir = "/home/nicolas.schneider/nicolas/master_thesis/CG_sim"  ##folder where the cg simulation data is stored
Cluster_sim.sel_charges = ["nacl_iso_charged/"]  #subfolders in which simulations will be searched
Cluster_sim.sel_peptides = ["ASP_20/"]           #subfolders in which simulations will be searched  
Cluster_sim.descriptor_selection = "name BB"     #MDA selection string for coarsegrained data
emap_template_folder = "template1"

#searching for subfolders and creating new Cluster_sim objects from the folder names - folder names have to match!
folders = []
cwd = os.getcwd()
os.chdir(cg_simulation_dir)
for charge in Cluster_sim.sel_charges:
    for peptide in Cluster_sim.sel_peptides:
        folders += glob.glob(charge + peptide + "*")
os.chdir(cwd)
folders = [folder.split("/") for folder in folders]

Cluster_sim.cg_sim_list = []
for folder in folders:
    print(folder)
    #try:
    Cluster_sim.cg_sim_list.append(Cluster_sim("CG_sim", *folder))
    #except:
    #    print("exception :", *folder)
    
Cluster_sim.evaluation_data_spec_directory = Cluster_sim.evaluation_data_base_directory
    
    #creating evaluation data directory based on the different systems it contains
attrs = ["sim_type_folder", "charge_folder", "peptide_folder"]
for attr in attrs:
    sim_attr = [getattr(o, attr) for o in Cluster_sim.cg_sim_list]
    set_attr = set(sim_attr)
    for string in set_attr:
        Cluster_sim.evaluation_data_spec_directory += string + "+"
    Cluster_sim.evaluation_data_spec_directory = Cluster_sim.evaluation_data_spec_directory[:-1] + "/"    

    #encodermap data path
Cluster_sim.e_map_path = os.path.join(Cluster_sim.evaluation_data_spec_directory, emap_template_folder)
if not os.path.exists(Cluster_sim.e_map_path):
    os.makedirs(Cluster_sim.e_map_path)
    
for sim in Cluster_sim.cg_sim_list:
    sim.gen_pair_dist_ts(Cluster_sim.descriptor_selection, sample_frames = len(sim.universe.trajectory))
    #sim.save_evaluation_data(Cluster_sim.evaluation_data_spec_directory + sim.cluster_folder)

Cluster_sim.high_d_data_concat_cg, Cluster_sim.frame_dict_concat_cg = Cluster_sim.concatenate_high_d_data(Cluster_sim.cg_sim_list)
#high_d_data_concat_norm_cg = Cluster_sim.norm_high_d_data(high_d_data_concat_cg)[0]
#Cluster_sim.save_evaluation_data(Cluster_sim, Cluster_sim.evaluation_data_spec_directory)

#print(Cluster_sim.evaluation_data_spec_directory)

['nacl_iso_neutral', 'ASP_10', 'cluster_19']
['nacl_iso_neutral', 'ASP_10', 'cluster_3']
['nacl_iso_neutral', 'ASP_10', 'cluster_4']
['nacl_iso_neutral', 'ASP_10', 'cluster_5']


In [ ]:
def_parameters = em.Parameters()
print("default parameters:")
print("center_cost_scale: ", def_parameters.center_cost_scale)
print("auto_cost_scale: ", def_parameters.auto_cost_scale)
print("distance_cost_scale: ", def_parameters.distance_cost_scale)
print("l2_reg_constant: ", def_parameters.l2_reg_constant)
print("learning_rate: ", def_parameters.learning_rate)

In [ ]:
e2e_dist_ts_concat_cg = []

for sim in Cluster_sim.cg_sim_list:
    e2e_dist_ts_concat_cg += sim.gen_end2end_dist_ts()

In [ ]:
#initializing encodermap
high_d_data_em = Cluster_sim.high_d_data_concat
sig_h = 45            
a_h = 20
b_h = a_h
a_l = math.ceil(a_h*2/len(high_d_data_em[0]))
b_l = b_h
parameters = em.Parameters()
parameters.main_path = Cluster_sim.e_map_path
parameters.n_steps = 5000
#parameters.n_neurons = [2048,2048,2048,2]
#parameters. activation_functions = ['', 'tanh', 'tanh', 'tanh', '']
parameters.dist_sig_parameters = (sig_h, a_h, b_h, 1, a_l, b_l)
parameters.periodicity = float("inf")
parameters.center_cost_scale = 0.03
parameters.auto_cost_scale = 24.#50.
parameters.l2_reg_constant = 0.039
parameters.learning_rate = 0.000023
parameters.distance_cost_scale = 27.
parameters.summary_step = max(1, parameters.n_steps/100)
parameters.checkpoint_step = 20

print(parameters.main_path)


#Cluster_sim.save_evaluation_data(Cluster_sim, parameters.main_path + "/evaluation_data/")
    

        
# if your data set is large you should not try to calculate 
# the pairwise distance histogram with the complete data. 
em.plot.distance_histogram(high_d_data_em[::100],
                           parameters.periodicity, 
                           parameters.dist_sig_parameters)

In [ ]:
#training emap
Cluster_sim.e_map = em.EncoderMap(parameters, train_data = high_d_data_em)
Cluster_sim.e_map.train()

In [ ]:
#opening tensorboard
%cd $parameters.main_path
process = subprocess.Popen(["tensorboard", "--logdir", ".", "--port", "6014"])

In [ ]:
#closing tensorboard
process.terminate()

%pwd

In [ ]:
#encoding high d data
projection_total = Cluster_sim.e_map.encode(high_d_data_em)

In [ ]:
#e2e dist map
%matplotlib notebook

fig, ax = plt.subplots(facecolor='white')

sc = ax.scatter(projection_total[:, 0], projection_total[:, 1], alpha=0.4, s = 6, marker = ".", c = e2e_dist_ts_concat_cg, cmap = "jet", zorder = 1, label = "CG_sim")

plt.legend()
plt.colorbar(sc, label="end-to-end distance Â")


plt.axis("off") 
plt.tight_layout()
plt.show()


In [ ]:
#free energy map
fig = plt.figure(figsize=(4.5, 4.5), dpi=200, facecolor='white')
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor("white")
ax.axis('off')

mplt.plot_free_energy(np.vstack(projection_total)[:,0], np.vstack(projection_total)[:,1], ax=ax,
                      cmap='nipy_spectral', ncontours=100, nbins=200)
#plt.savefig('savepath.png', bbox_inches="tight")
plt.show()

In [ ]:
vars(Cluster_sim)

In [ ]:
#saving class state --> next load it for UA_evaluation
obj_name = "ASP10_neutral_1" + ".pickle"
Cluster_sim.save_state(os.path.join("/home/nicolas.schneider/nicolas/master_thesis/serialized_objects/", obj_name))